# Procesamiento de eventos de incendios CONAF para dominios WRF

Este notebook desarrolla un flujo de procesamiento de datos de incendios forestales de la base oficial de CONAF (procesada por el CR2), filtrados espacial y temporalmente para análisis en dominios del modelo WRF (D03 y D04).

---

## 📜 Resumen de las celdas

### 1️⃣ Generación de base nacional
- Se leen todos los archivos por temporada desde la carpeta `dataverse_files/data/`, donde cada temporada está en formato CSV (`<temporada>.csv`) con delimitador `|`.
- Se concatenan en un único `DataFrame` con todos los eventos registrados en Chile.
- El resultado se guarda en el archivo **`datos-CONAF-PAIS.pkl`**.

### 2️⃣ Filtro por Región de Valparaíso
- A partir de `datos-CONAF-PAIS.pkl`, se filtran los eventos cuya columna `Región` corresponde a **"Valparaíso"** (ignorando mayúsculas y espacios).
- El resultado se guarda como **`datos-CONAF-VREGION.pkl`**.

### 3️⃣ Filtro espacial para dominios WRF D03 y D04
- Se cargan las grillas de superficie de WRF desde los archivos:
  - **`grilla_WRF-Superficie-D03.pkl`**
  - **`grilla_WRF-Superficie-D04.pkl`**
- Se convierten a `GeoDataFrame` y se genera el polígono envolvente de cada dominio.
- Se filtran los eventos de `datos-CONAF-VREGION.pkl` que caen dentro de cada dominio.
- Se generan y guardan los siguientes archivos:
  - **`datos-CONAF-VREGION-D03_geo.pkl`** → GeoDataFrame con geometría para D03.
  - **`datos-CONAF-VREGION-D03.pkl`** → DataFrame tabular para D03.
  - **`datos-CONAF-VREGION-D04_geo.pkl`** → GeoDataFrame con geometría para D04.
  - **`datos-CONAF-VREGION-D04.pkl`** → DataFrame tabular para D04.

### 4️⃣ Inspección visual con Folium
- Se crean mapas interactivos con Folium para eventos **≥10 ha** a partir de:
  - **`datos-CONAF-VREGION-D03_geo.pkl`**
  - **`datos-CONAF-VREGION-D04_geo.pkl`**
- Se utilizan marcadores circulares con tamaño proporcional a la superficie quemada y `popup` con información de Comuna, Fecha, Superficie y Causa.
- Los mapas pueden guardarse en:
  - **`mapa_D03_incendios_mayor10ha.html`**
  - **`mapa_D04_incendios_mayor10ha.html`**
 

### 5️⃣ Resumen estadístico
- A partir de los archivos **`datos-CONAF-VREGION-D03.pkl`** y **`datos-CONAF-VREGION-D04.pkl`**:
  - Se calcula el número total de eventos.
  - Se determina el rango temporal de fechas.
  - Se listan las temporadas disponibles.
  - Se calcula la superficie total, promedio y máxima.
- El análisis se repite solo para eventos **≥10 ha**.

---

## Archivos de entrada necesarios

- Carpeta con datos de incendios por temporada


In [1]:
import pandas as pd
import os

# === 1. Ruta base donde están las carpetas por temporada ===
# Cada carpeta contiene un archivo CSV con datos de incendios para esa temporada
ruta_base = r'dataverse_files/data'

# === 2. Generar lista de temporadas ===
# Formato: "2002-2003" → "0203", pero aquí usamos '200203', '200304', etc. según el formato dado
# En este caso: 2002 hasta 2019 (2002-2003, 2003-2004, ... 2019-2020)
temporadas = [f"{a}{a+1:02d}" for a in range(2002, 2020)]

# === 3. Lista para almacenar los DataFrames de cada temporada ===
lista_df = []

# === 4. Recorrer todas las temporadas y leer cada archivo CSV ===
for temporada in temporadas:
    # Ruta al archivo CSV correspondiente a la temporada
    ruta_csv = os.path.join(ruta_base, temporada, f"{temporada}.csv")
    
    try:
        # Leer el archivo CSV
        # Usamos '|' como delimitador y codificación UTF-8
        df_temp = pd.read_csv(ruta_csv, delimiter='|', encoding='utf-8', engine='python')
        
        # Agregar columna con la temporada para mantener la referencia temporal
        df_temp['temporada'] = temporada
        
        # Agregar el DataFrame temporal a la lista
        lista_df.append(df_temp)
    
    except Exception as e:
        # Si hay un error (archivo no encontrado, problema de formato, etc.), lo mostramos
        print(f"Error al leer {ruta_csv}: {e}")

# === 5. Combinar todos los DataFrames en uno solo ===
df_incendios = pd.concat(lista_df, ignore_index=True)

# === 6. Mostrar resumen de datos combinados ===
print(f"Total de filas combinadas: {len(df_incendios)}")
print("Columnas disponibles en el DataFrame:")
print(df_incendios.columns)

# === 7. Guardar DataFrame combinado como archivo pickle ===
# Esto facilita su carga rápida en futuros análisis
df_incendios.to_pickle("datos-CONAF-PAIS.pkl")


Total de filas combinadas: 109985
Columnas disponibles en el DataFrame:
Index(['Región', 'Provincia', 'Comuna', 'Temporada', 'Nombre', 'Fecha',
       'Hora inicio', 'Duración (minutos)', 'Alerta', 'Escenario', 'Causa',
       'Superficie quemada: Pino A [ha]', 'Superficie quemada: Pino B [ha]',
       'Superficie quemada: Pino C [ha]', 'Superficie quemada: Eucalípto [ha]',
       'Superficie quemada: Otras plantas [ha]',
       'Superficie quemada: Arbolado [ha]',
       'Superficie quemada: Matorral [ha]',
       'Superficie quemada: Pastizal [ha]',
       'Superficie quemada: Agrícola [ha]',
       'Superficie quemada: Desechos [ha]', 'Superficie quemada total [ha]',
       'Latitud', 'Longitud', 'Datum', 'temporada'],
      dtype='object')


In [ ]:
# === 2. Filtrar eventos para la Región de Valparaíso ===

# Filtrar registros donde la columna 'Región' sea exactamente 'Valparaíso' (ignorando mayúsculas/espacios)
df_valparaiso = df_incendios[
    df_incendios['Región'].str.strip().str.lower() == 'valparaíso'
].copy()

# Resumen del filtro
print(f"Cantidad de incendios en la Región de Valparaíso: {len(df_valparaiso)}")

# Comunas presentes en la región filtrada
print("Comunas en la Región de Valparaíso:")
print(df_valparaiso['Comuna'].unique())

# Guardar en formato pickle
df_valparaiso.to_pickle("datos-CONAF-VREGION.pkl")


In [5]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import box

# === 3. Filtrar eventos dentro de D03 y D04 (robusto a grillas sin 'geometry') ===

# 0) Helper: convertir DF de grilla a GeoDataFrame (crea geometry si falta)
def to_gdf_grid(df, crs="EPSG:4326"):
    if isinstance(df, gpd.GeoDataFrame) and df.geometry.name in df.columns:
        gdf = df.copy()
        if gdf.crs is None: gdf.set_crs(crs, inplace=True)
        return gdf

    df = df.copy()
    # posibles nombres de columnas lon/lat
    lon_candidates = ["lon", "Lon", "LON", "Longitud", "longitud", "x", "X"]
    lat_candidates = ["lat", "Lat", "LAT", "Latitud", "latitud", "y", "Y"]

    lon_col = next((c for c in lon_candidates if c in df.columns), None)
    lat_col = next((c for c in lat_candidates if c in df.columns), None)

    if lon_col is None or lat_col is None:
        raise ValueError(
            "No se encontró 'geometry' ni pares lon/lat en la grilla. "
            f"Columnas disponibles: {list(df.columns)}"
        )

    # asegurar numéricos
    df[lon_col] = pd.to_numeric(df[lon_col], errors="coerce")
    df[lat_col] = pd.to_numeric(df[lat_col], errors="coerce")
    df = df.dropna(subset=[lon_col, lat_col])

    gdf = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df[lon_col], df[lat_col]),
        crs=crs
    )
    return gdf

# 1) Cargar incendios Valparaíso (usa nombres correctos: 'Longitud', 'Latitud')
df_valpo = pd.read_pickle("datos-CONAF-VREGION.pkl").copy()
df_valpo["Longitud"] = pd.to_numeric(df_valpo["Longitud"], errors="coerce")
df_valpo["Latitud"]  = pd.to_numeric(df_valpo["Latitud"],  errors="coerce")
df_valpo = df_valpo.dropna(subset=["Longitud", "Latitud"])

gdf_valpo = gpd.GeoDataFrame(
    df_valpo,
    geometry=gpd.points_from_xy(df_valpo["Longitud"], df_valpo["Latitud"]),
    crs="EPSG:4326"
)

# 2) Cargar grillas D03 y D04 (con pandas) y garantizar GeoDataFrame
df_d03 = pd.read_pickle("grilla_WRF-Superficie-D03.pkl")
df_d04 = pd.read_pickle("grilla_WRF-Superficie-D04.pkl")

gdf_d03 = to_gdf_grid(df_d03, crs="EPSG:4326")
gdf_d04 = to_gdf_grid(df_d04, crs="EPSG:4326")

# 3) Bounding box de cada dominio
minx, miny, maxx, maxy = gdf_d03.total_bounds
poly_d03 = box(minx, miny, maxx, maxy)

minx, miny, maxx, maxy = gdf_d04.total_bounds
poly_d04 = box(minx, miny, maxx, maxy)

# 4) Filtrar eventos dentro de cada dominio
gdf_valpo_d03 = gdf_valpo[gdf_valpo.geometry.within(poly_d03)].copy()
gdf_valpo_d04 = gdf_valpo[gdf_valpo.geometry.within(poly_d04)].copy()

# 5) Resumen
print(f"Incendios en D03: {len(gdf_valpo_d03)}")
print(f"Incendios en D04: {len(gdf_valpo_d04)}")

# 6) Guardar versiones GeoDataFrame (con geometría)
gdf_valpo_d03.to_pickle("datos-CONAF-VREGION-D03_geo.pkl")
gdf_valpo_d04.to_pickle("datos-CONAF-VREGION-D04_geo.pkl")

# 7) Guardar versiones pandas (sin geometría)
gdf_valpo_d03.drop(columns="geometry").to_pickle("datos-CONAF-VREGION-D03.pkl")
gdf_valpo_d04.drop(columns="geometry").to_pickle("datos-CONAF-VREGION-D04.pkl")


Incendios en D03: 9881
Incendios en D04: 5339


In [7]:
# === 4. Inspección rápida con folium: eventos > 10 ha en D03 y D04 ===
import pandas as pd
import geopandas as gpd
import folium

def ensure_gdf(path, crs="EPSG:4326"):
    """Carga un pickle y asegura que sea GeoDataFrame con CRS."""
    df = pd.read_pickle(path)
    if not isinstance(df, gpd.GeoDataFrame):
        if "geometry" in df.columns:
            df = gpd.GeoDataFrame(df, geometry="geometry", crs=crs)
        else:
            raise ValueError(f"El archivo {path} no contiene geometría.")
    elif df.crs is None:
        df = df.set_crs(crs)
    return df

# Cargar datasets
gdf_d03 = ensure_gdf("datos-CONAF-VREGION-D03_geo.pkl")
gdf_d04 = ensure_gdf("datos-CONAF-VREGION-D04_geo.pkl")

# Nombre exacto de la columna de superficie
col_sup = "Superficie quemada total [ha]"

# Verificar que la columna exista
for name, gdf in [("D03", gdf_d03), ("D04", gdf_d04)]:
    if col_sup not in gdf.columns:
        raise KeyError(f"La columna '{col_sup}' no está en {name}. Columnas: {list(gdf.columns)}")

# Filtrar por superficie > 10 ha
gdf_d03_10 = gdf_d03[gdf_d03[col_sup] > 10].copy()
gdf_d04_10 = gdf_d04[gdf_d04[col_sup] > 10].copy()

print(f"D03: {len(gdf_d03_10)} eventos > 10 ha")
print(f"D04: {len(gdf_d04_10)} eventos > 10 ha")

def make_map(gdf, zoom=8, tiles="CartoDB positron"):
    if len(gdf) == 0:
        center = [-33.0, -71.6]  # fallback
    else:
        center = [gdf.geometry.y.mean(), gdf.geometry.x.mean()]
    m = folium.Map(location=center, zoom_start=zoom, tiles=tiles)
    for _, row in gdf.iterrows():
        area = row[col_sup]
        radius = 4 + min(12, (area ** 0.5) / 2)  # tamaño relativo
        popup_html = (
            f"<b>Comuna:</b> {row['Comuna']}<br>"
            f"<b>Fecha:</b> {row['Fecha']}<br>"
            f"<b>Superficie:</b> {area:.1f} ha<br>"
            f"<b>Causa:</b> {row['Causa']}"
        )
        folium.CircleMarker(
            location=[row.geometry.y, row.geometry.x],
            radius=radius,
            fill=True,
            fill_opacity=0.7,
            opacity=0.8,
            popup=folium.Popup(popup_html, max_width=280),
        ).add_to(m)
    return m

# Mapas
m_d03 = make_map(gdf_d03_10)
m_d04 = make_map(gdf_d04_10)

m_d03  # mapa D03
# m_d03.save("mapa_D03_incendios_mayor10ha.html")

m_d04  # mapa D04
# m_d04.save("mapa_D04_incendios_mayor10ha.html")


D03: 560 eventos > 10 ha
D04: 143 eventos > 10 ha


In [8]:
# === 5. Resumen de eventos por dominio ===
import pandas as pd

# Cargar datos (versión tabular, sin geometría)
df_d03 = pd.read_pickle("datos-CONAF-VREGION-D03.pkl")
df_d04 = pd.read_pickle("datos-CONAF-VREGION-D04.pkl")

def resumen_eventos(nombre, df):
    print(f"\n=== {nombre} ===")
    print(f"Total de eventos: {len(df):,}")
    
    # Rango de fechas
    if "Fecha" in df.columns:
        fechas = pd.to_datetime(df["Fecha"], errors="coerce")
        fecha_min = fechas.min()
        fecha_max = fechas.max()
        print(f"Rango de fechas: {fecha_min.date()} → {fecha_max.date()}")
    
    # Temporadas
    if "temporada" in df.columns:
        temporadas_unicas = sorted(df["temporada"].unique())
        print(f"Total de temporadas: {len(temporadas_unicas)}")
        print(f"Temporadas: {', '.join(map(str, temporadas_unicas))}")
    
    # Estadísticas rápidas de superficie
    col_sup = "Superficie quemada total [ha]"
    if col_sup in df.columns:
        sup_total = df[col_sup].sum()
        sup_media = df[col_sup].mean()
        sup_max = df[col_sup].max()
        print(f"Superficie total quemada: {sup_total:,.1f} ha")
        print(f"Superficie promedio: {sup_media:,.1f} ha")
        print(f"Mayor superficie registrada: {sup_max:,.1f} ha")

# Mostrar resumen para cada dominio
resumen_eventos("Dominio D03", df_d03)
resumen_eventos("Dominio D04", df_d04)



=== Dominio D03 ===
Total de eventos: 9,881
Rango de fechas: 2002-10-04 → 2020-06-11
Total de temporadas: 18
Temporadas: 20022003, 20032004, 20042005, 20052006, 20062007, 20072008, 20082009, 20092010, 20102011, 20112012, 20122013, 20132014, 20142015, 20152016, 20162017, 20172018, 20182019, 20192020
Superficie total quemada: 84,952.3 ha
Superficie promedio: 8.6 ha
Mayor superficie registrada: 3,740.0 ha

=== Dominio D04 ===
Total de eventos: 5,339
Rango de fechas: 2002-10-04 → 2020-05-30
Total de temporadas: 18
Temporadas: 20022003, 20032004, 20042005, 20052006, 20062007, 20072008, 20082009, 20092010, 20102011, 20112012, 20122013, 20132014, 20142015, 20152016, 20162017, 20172018, 20182019, 20192020
Superficie total quemada: 18,980.7 ha
Superficie promedio: 3.6 ha
Mayor superficie registrada: 1,324.2 ha


In [9]:
# === 5. Resumen de eventos >= 10 ha por dominio ===
import pandas as pd

# Cargar datos (versión tabular, sin geometría)
df_d03 = pd.read_pickle("datos-CONAF-VREGION-D03.pkl")
df_d04 = pd.read_pickle("datos-CONAF-VREGION-D04.pkl")

# Columna de superficie
col_sup = "Superficie quemada total [ha]"

# Filtrar eventos >= 10 ha
df_d03_10 = df_d03[df_d03[col_sup] >= 10].copy()
df_d04_10 = df_d04[df_d04[col_sup] >= 10].copy()

def resumen_eventos(nombre, df):
    print(f"\n=== {nombre} (>= 10 ha) ===")
    print(f"Total de eventos: {len(df):,}")
    
    # Rango de fechas
    if "Fecha" in df.columns:
        fechas = pd.to_datetime(df["Fecha"], errors="coerce")
        fecha_min = fechas.min()
        fecha_max = fechas.max()
        print(f"Rango de fechas: {fecha_min.date()} → {fecha_max.date()}")
    
    # Temporadas
    if "temporada" in df.columns:
        temporadas_unicas = sorted(df["temporada"].unique())
        print(f"Total de temporadas: {len(temporadas_unicas)}")
        print(f"Temporadas: {', '.join(map(str, temporadas_unicas))}")
    
    # Estadísticas rápidas de superficie
    if col_sup in df.columns:
        sup_total = df[col_sup].sum()
        sup_media = df[col_sup].mean()
        sup_max = df[col_sup].max()
        print(f"Superficie total quemada: {sup_total:,.1f} ha")
        print(f"Superficie promedio: {sup_media:,.1f} ha")
        print(f"Mayor superficie registrada: {sup_max:,.1f} ha")

# Mostrar resumen para cada dominio
resumen_eventos("Dominio D03", df_d03_10)
resumen_eventos("Dominio D04", df_d04_10)



=== Dominio D03 (>= 10 ha) ===
Total de eventos: 595
Rango de fechas: 2002-11-09 → 2020-05-14
Total de temporadas: 18
Temporadas: 20022003, 20032004, 20042005, 20052006, 20062007, 20072008, 20082009, 20092010, 20102011, 20112012, 20122013, 20132014, 20142015, 20152016, 20162017, 20172018, 20182019, 20192020
Superficie total quemada: 76,987.6 ha
Superficie promedio: 129.4 ha
Mayor superficie registrada: 3,740.0 ha

=== Dominio D04 (>= 10 ha) ===
Total de eventos: 151
Rango de fechas: 2002-11-20 → 2020-05-05
Total de temporadas: 17
Temporadas: 20022003, 20032004, 20042005, 20052006, 20062007, 20072008, 20082009, 20092010, 20102011, 20112012, 20122013, 20132014, 20142015, 20162017, 20172018, 20182019, 20192020
Superficie total quemada: 15,916.4 ha
Superficie promedio: 105.4 ha
Mayor superficie registrada: 1,324.2 ha
